<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [60]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')
from collections import Counter

In [61]:
# Данные для подключения к базе данных 

In [62]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [63]:
# текст запроса
query_3_1 = '''select 
                    count(id) as vacancies_count
                from 
                    public.vacancies
'''

In [64]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, connection)
print(f'В базе данных содержится {df_3_1.iloc[0,0]} вакансий')

В базе данных содержится 49197 вакансий


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [65]:
# текст запроса
query_3_2 = '''select 
                    count(id) as employers_count
                from 
                    public.employers 
'''

In [66]:
# результат запроса
df_3_2 = pd.read_sql_query(query_3_2, connection)
print(f'В базе данных представлены сведения о {df_3_2.iloc[0,0]} работодателе')

В базе данных представлены сведения о 23501 работодателе


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [67]:
# текст запроса
query_3_3 = '''select
                    count(id) as areas_count
                from
                    public.areas 
'''

In [68]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, connection)
print(f'В базе данных содержатся сведения о {df_3_3.iloc[0,0]} регионах')

В базе данных содержатся сведения о 1362 регионах


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [69]:
# текст запроса
query_3_4 = '''select
                    count(id) as industries_count
                from
                    public.industries 
'''

In [70]:
# результат запроса
df_3_4 = pd.read_sql_query(query_3_4, connection)
print(f'В базе данных содержатся сведения о {df_3_4.iloc[0,0]} сферах деятельности работодателей')

В базе данных содержатся сведения о 294 сферах деятельности работодателей


***

### Выводы по предварительному анализу данных:

Исследуемая база данных содержит сведения о **49197 вакансиях** для различных специалистов IT-сферы.

В базе зарегистрирован **23501 работодатель**, но активные вакансии есть не от каждого из них. Подробный разбор работодателей я привожу в дополнительном блоке в конце проекта.

В базу занесено **1362 региона**, в которых могут быть размещены вакансии. Но активные вакансии есть не в каждом регионе. Кроме того, список регионов не унифицирован и не имеет разбивки от крупных территориальных субъектов к мелким (страна - регион - населенный пункт). Поэтому в базе отдельными уникальными регионами считаются, например, Москва, Московская область, Россия. А это усложняет анализ данных и без детальной проработки данных по регионам может внести в результаты исследования существенные искажения.


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [71]:
# текст запроса
query_4_1 = '''select
                    a.name area, 
                    count(v.id) as cnt
                from 
                    public.vacancies v 
                    join public.areas a on v.area_id = a.id
                group by 
                    a.name
                order by 
                    count(v.id) desc
'''

In [72]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
display(df_4_1.head())
print(f'Наибольшее число вакансий размещено в следующих регионах: {df_4_1.iloc[0,0]}, {df_4_1.iloc[1,0]}, {df_4_1.iloc[2,0]}, {df_4_1.iloc[3,0]}, {df_4_1.iloc[4,0]}.')

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


Наибольшее число вакансий размещено в следующих регионах: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы.


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [73]:
# текст запроса
query_4_2 = '''select
                    count(id)
                from 
                    public.vacancies
                where 
                    salary_from is not null 
                    or salary_to is not null
'''

In [74]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, connection)
print(f'Заработная плата указана в {df_4_2.iloc[0,0]} вакансиях из {df_3_1.iloc[0,0]}, содержащихся в базе данных')

Заработная плата указана в 24073 вакансиях из 49197, содержащихся в базе данных


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [75]:
# текст запроса
query_4_3 = '''select 
                    round(avg(salary_from)) as avg_salary_from, 
                    round(avg(salary_to)) as avg_salary_to
                from 
                    public.vacancies
'''

In [76]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
print(f'Средние значения зарплатной вилки по всем вакансиям находятся в диапазоне от {df_4_3.iloc[0,0]} до {df_4_3.iloc[0,1]} рублей')

Средние значения зарплатной вилки по всем вакансиям находятся в диапазоне от 71065.0 до 110537.0 рублей


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [77]:
# текст запроса
query_4_4 = '''select 
                    schedule, 
                    employment, 
                    count(id) as vacancies_count
                from 
                    public.vacancies
                group by 
                    schedule, 
                    employment
                order by 
                    count(id) desc
'''

In [78]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
display(df_4_4)
print(f'Больше всего вакансий в базе для соискателей, которым подходит "{df_4_4.iloc[0,1]}" и график работы "{df_4_4.iloc[0,0]}" или "{df_4_4.iloc[1,0]}"')

,schedule,employment,vacancies_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


Больше всего вакансий в базе для соискателей, которым подходит "Полная занятость" и график работы "Полный день" или "Удаленная работа"


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [79]:
# текст запроса
query_4_5 = '''select
                    experience, 
                    count(id) vacancies_count
                from 
                    public.vacancies
                group by 
                    experience
                order by 
                    count(id)
'''

In [80]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
display(df_4_5)
print(f'Больше всего вакансий в базе для соискателей с опытом работы "{df_4_5.iloc[3,0]}", и меньше всего - для специалистов с опытом работы "{df_4_5.iloc[0,0]}"')

,experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


Больше всего вакансий в базе для соискателей с опытом работы "От 1 года до 3 лет", и меньше всего - для специалистов с опытом работы "Более 6 лет"


***

### Выводы по детальному анализу вакансий:

Анализ вакансий показал, что большинство доступных вакансий размещены в крупнейших городах России (Москва, Санкт-Петербург и городах-миллионники), а также в столицах стран ближнего зарубежья. Это вполне ожидаемо, так как количество рабочих мест напрямую зависит от численности населения в регионе, а в крупных регионах IT-сфера наиболее развита, так как там сосредоточены центральные представительства всех лидеров отрасли. 
Можно также отметить, что портал Headhanter популярен при размещении вакансий и поиске работы не только в России, но и за рубежом.

Подавляющее большинство работодателей (более 93%) ищут сотрудников **на полную занятость**. При этом лишь 15% из них готовы взять сотрудника "на удаленку". 

Виден наибольший спрос на сотрудников **с опытом работы** (более 85% от всех вакансий). 
Но и начинающие IT-специалисты имеют выбор из 7197 вакансий для старта в карьере. Среди вакансий для новичков есть различные варианты со сатжировкой на производстве.

Заработная плата в IT-сфере имеет большой разброс, в размещенных в базе вакансиях указаны цифры от 30 рублей до 1 млн рублей. Однако, это крайние значения и скорее всего их можно считать выбросами. Средняя зарплатная вилка по отрасли составляет **от 71065 до 110537 рублей**.


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [81]:
# текст запроса
query_5_1 = '''select
                    e.name,
                    count(v.id) as vacancies_count
                from 
                    public.vacancies v
                    join public.employers e on v.employer_id = e.id
                group by 
                    e.name
                order by 
                    count(v.id) desc
'''

In [82]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
display(df_5_1.head())
print(f'На первом месте среди работодателей по числу вакансий находится компания "{df_5_1.iloc[0,0]}", на 5-м месте - "{df_5_1.iloc[4,0]}"')

,name,vacancies_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


На первом месте среди работодателей по числу вакансий находится компания "Яндекс", на 5-м месте - "Газпром нефть"


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [83]:
# текст запроса
query_5_2 = f'''select 
                    a.name as area, 
                    count(v.id) as vacancies_count, 
                    count(e.id) as employers_count
                from 
                    public.areas a
                    left join public.vacancies v on a.id = v.area_id
                    left join public.employers e on a.id = e.area
                group by 
                    a.id
                having 
                    count(v.id) = 0
                order by 
                    count(e.id) desc
'''

In [84]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
display(df_5_2.head())
print(f'Наибольшее число работодателей, от которых нет вакансий ({df_5_2.iloc[0,2]} компаний), относятся к обобщенному региону "{df_5_2.iloc[0,0]}"')

,area,vacancies_count,employers_count
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18


Наибольшее число работодателей, от которых нет вакансий (410 компаний), относятся к обобщенному региону "Россия"


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [85]:
# текст запроса
query_5_3 = '''select 
                    e.name as employer_name, 
                    count(distinct v.area_id) as areas_count
                from 
                    public.vacancies v
                    join public.employers e on v.employer_id = e.id
                group by 
                    e.name
                order by
                    count(distinct v.area_id) desc
'''

In [86]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
display(df_5_3.head(10))
print(f'Компания {df_5_3.iloc[0,0]} лидирует среди работодателей по числу регионов, в которых размещены ее вакансии')

,employer_name,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,Почта России,48
9,КРОН,48


Компания Яндекс лидирует среди работодателей по числу регионов, в которых размещены ее вакансии


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [87]:
# текст запроса
query_5_4 = '''select 
                    count(e.id) as count_null_industry
                from 
                    public.employers e
                    left join public.employers_industries ei on e.id = ei.employer_id
                where 
                    ei.industry_id is null
'''

In [88]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
print(f'В базе данных содержится информация о {df_5_4.iloc[0,0]} работодателях, у которых не указана сфера деятельности')

В базе данных содержится информация о 8419 работодателях, у которых не указана сфера деятельности


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [89]:
# текст запроса
query_5_5 = '''select 
                    e.name as employer_name, 
                    count(ei.industry_id) as industries_count
                from 
                    public.employers e
                    left join public.employers_industries ei on e.id = ei.employer_id
                group by 
                    e.id
                having
                    count(ei.industry_id) = 4
                order by 
                    e.name
'''

In [90]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
display(df_5_5.head())
print(f'Среди работодателей, у которых указано 4 сферы деятельности, на 3-м месте (по алфавиту) находится компания "{df_5_5.iloc[2,0]}"')

,employer_name,industries_count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4


Среди работодателей, у которых указано 4 сферы деятельности, на 3-м месте (по алфавиту) находится компания "2ГИС"


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [91]:
# текст запроса
query_5_6 = '''select 
                    count(ei.employer_id) as employers_count
                from 
                    public.employers_industries ei
                    left join public.industries i on ei.industry_id = i.id
                where 
                    i.name = 'Разработка программного обеспечения'
'''

In [92]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
print(f'В базе данных имеется информация о {df_5_6.iloc[0,0]} работодателях, одной из сфер деятельности которых указана "Разработка программного обеспечения"')

В базе данных имеется информация о 3553 работодателях, одной из сфер деятельности которых указана "Разработка программного обеспечения"


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [93]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
million_city_df = pd.read_html(url)[1]
million_city_df = million_city_df['Город'].replace('Воронеж[a]', 'Воронеж')
million_city_tuple = tuple(million_city_df)
print(million_city_tuple)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Красноярск', 'Челябинск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [94]:
# текст запроса
query_5_7 = f'''select 
                    a.name as million_city_name, 
                    count(v.id) as vacancies_count
                from 
                    public.vacancies v
                    left join public.areas a on v.area_id = a.id
                    left join public.employers e on v.employer_id = e.id
                where 
                    e.name = 'Яндекс' 
                    and a.name in {million_city_tuple}
                group by 
                    a.id
                
                union all
                
                select 
                    'Total', 
                    count(v.id)
                from 
                    public.vacancies v
                    left join public.areas a on v.area_id = a.id
                    left join public.employers e on v.employer_id = e.id
                where 
                    e.name = 'Яндекс' 
                    and a.name in {million_city_tuple}
                
                order by 2
'''

In [95]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
display(df_5_7)
print(f'Компанией Яндекс размещено {df_5_7.iloc[16,1]} вакансий в городах-миллионниках России')

,million_city_name,vacancies_count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


Компанией Яндекс размещено 485 вакансий в городах-миллионниках России


***

### Выводы по анализу работодателей:

Анализ работодателей показал, что в базе зарегистрирован **23501 работодатель**.

В первую очередь следует выделить крупнейших игроков рынка - крупные межрегиональные и международные компании, которым требуется огромный штат сотрудников. 
Привожу ТОП 10 компаний по числу размещенных вакансий: Яндекс, Ростелеком, Тинькофф, СБЕР, Газпром нефть, ИК СИБИНТЕК, МТС, DataArt, Совкомбанк Технологии, Первый Бит.

Вакансии от лидеров рынка предоставлены в большинстве крупных регионов России и многих иностранных представительствах компании. Большой штат сотрудников динамичен и постоянно требует новых специалистов для покрытия потребностей этих компаний. 

Также замечу, что далеко не у всех занесенных в базу работодателей есть активные размещенные вакансии. Компаний, от которых в данный момент есть вакансии - 63% от общего числа работодателей.

Подробный разбор работодателей я привожу в дополнительном блоке в конце проекта.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [96]:
# текст запроса
query_6_1 = '''select 
                    count(id) as vacancies_count
                from 
                    public.vacancies
                where 
                    name ilike '%data%' 
                    or name ilike '%данн%' 
'''

In [97]:
# результат запроса
df_6_1 = pd.read_sql_query(query_6_1, connection)
print(f'В базе данных имеется {df_6_1.iloc[0,0]} вакансия, имеющая отношение к работе с данными')

В базе данных имеется 1771 вакансия, имеющая отношение к работе с данными


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [98]:
# текст запроса
query_6_2 = '''select 
                    count(id) as vacancies_count
                from 
                    public.vacancies
                where 
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%' 
                    or name ilike '%исследователь данных%' 
                    or (name like '%ML%' and name not like '%HTML%') 
                    or name ilike '%machine learning%' 
                    or name ilike '%машинн%обучен%')
                    and 
                    (name ilike '%junior%' 
                    or experience = 'Нет опыта' 
                    or employment = 'Стажировка') 
'''

In [99]:
# результат запроса
df_6_2 = pd.read_sql_query(query_6_2, connection)
print(f'В базе данных имеется {df_6_2.iloc[0,0]} вакансия, подходящая для начинающего дата-сайентиста')

В базе данных имеется 51 вакансия, подходящая для начинающего дата-сайентиста


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [100]:
# текст запроса
query_6_3 = '''select 
                    count(id)  as vacancies_count
                from 
                    public.vacancies
                where 
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%' 
                    or name ilike '%исследователь данных%' 
                    or (name like '%ML%' and name not like '%HTML%') 
                    or name ilike '%machine learning%' 
                    or name ilike '%машинн%обучен%')
                    and 
                    (key_skills ilike '%SQL%' 
                    or key_skills ilike '%postgres%') 
'''

In [101]:
# результат запроса
df_6_3 = pd.read_sql_query(query_6_3, connection)
print(f'В базе данных имеется {df_6_3.iloc[0,0]} вакансия для DS, где в качестве ключевого навыка указан SQL или postgres')

В базе данных имеется 201 вакансия для DS, где в качестве ключевого навыка указан SQL или postgres


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [102]:
# текст запроса
query_6_4 = '''select 
                    count(id)  as vacancies_count
                from 
                    public.vacancies
                where 
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%' 
                    or name ilike '%исследователь данных%' 
                    or (name like '%ML%' and name not like '%HTML%') 
                    or name ilike '%machine learning%' 
                    or name ilike '%машинн%обучен%')
                    and key_skills ilike '%python%' 
'''

In [103]:
# результат запроса
df_6_4 = pd.read_sql_query(query_6_4, connection)
print(f'В базе данных имеется {df_6_4.iloc[0,0]} вакансия для DS, где в качестве ключевого навыка указан Python')

В базе данных имеется 351 вакансия для DS, где в качестве ключевого навыка указан Python


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [104]:
# текст запроса
query_6_5 = '''select 
                    round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2)
                from 
                    public.vacancies
                where 
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%' 
                    or name ilike '%исследователь данных%' 
                    or (name like '%ML%' and name not like '%HTML%') 
                    or name ilike '%machine learning%' 
                    or name ilike '%машинн%обучен%') 
'''

In [105]:
# результат запроса
df_6_5 = pd.read_sql_query(query_6_5, connection)
print(f'В подходящих для DS вакансиях в среднем указывается по {df_6_5.iloc[0,0]} ключевых навыков')

В подходящих для DS вакансиях в среднем указывается по 6.41 ключевых навыков


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [106]:
# текст запроса
query_6_6 = '''select 
                    experience, 
                    avg((coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from))/2)
                from 
                    public.vacancies
                where 
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%' 
                    or name ilike '%исследователь данных%' 
                    or (name like '%ML%' and name not like '%HTML%') 
                    or name ilike '%machine learning%' 
                    or name ilike '%машинн%обучен%')
                    and 
                    (salary_from is not null 
                    or salary_to is not null)
                group by 
                    experience 
'''

In [107]:
# результат запроса
df_6_6 = pd.read_sql_query(query_6_6, connection)
display(df_6_6)
avg_salary_3_6_year = round(df_6_6.iloc[2,1])
print(f'Средняя заработная плата для дата-сайентиста с опытом работы от 3 до 6 лет составляет {avg_salary_3_6_year} рублей')

,experience,avg
0,Нет опыта,74642.857143
1,От 1 года до 3 лет,139674.750000
2,От 3 до 6 лет,243114.666667


Средняя заработная плата для дата-сайентиста с опытом работы от 3 до 6 лет составляет 243115 рублей


***

### Выводы по предметному анализу:

Анализ вакансий в предоставленной базе данных показал, что на долю специалистов по работе с данными приходится 1771 вакансия - 3,6% от общего числа вакансий в базе. Не так много, но и не так мало, учитывая, насколько разносторонняя IT-сфера в целом.

Заработная плата DS-специалистов сильно варьируется от опыта работы, но даже для новичка доход более чем достойный (средняя зарплата составляет 74642 рубля). Для специалистов с опытом, средняя заработная плата увеличивается в разы: с опытом от 1 года она составляет уже 139675 рублей, а с опытом от 3 лет досигает 243115 рублей. 
В вакансиях для специалистов с опытом работы более 6 лет зарплата в вакансиях вообще не указывается, но можно предположить, что она еще выше и обговаривается непосредственно на собеседовании.

В сравнении со средней вилкой заработной платы по IT-отрасли вцелом (которую высчитывали ранее), статус специалистов по работе с данными ценится и предлагаемые для них вакансии оплачиваются гораздо выше.

Подробный разбор по востребованным ключевым навыкам IT-специалиста я привожу в дополнительном блоке в конце проекта.

# Общий вывод по проекту

Исходя из проведенного анализа базы ваканий можно сделать вывод, что специалисты IT-сферы востребованы на рынке труда, а уровень предлагаемой заработной платы в данной сфере значитально выше средних значений по России.

Так, согласно полученным из базы расчетам средняя зарплатная вилка в IT-сфере находится в диапазоне от 71065 до 110537 рублей, тогда как средняя зарплата по России по данным Росстата за 2022 год составляла 64191 рублей. Для DS-специалистов средний уровень зарплаты еще выше и значительно возрастает в зависимости от опыта работы.

Рынок труда имеет явных лидеров-работодателей (например, Яндекс, Ростелеком, Сбербанк, Газпром, МТС), которые предлагают сотни вакансий для IT-специалистов всех профилей. Работа в таких компаниях престижна и дает широкие возможности для профессионального развития специалиста и продвижения его по карьерной лестнице.
Но и в малом бизнесе без IT-специалистов не обойтись, более половины вакансий в базе от таких компаний.

Портал Headhunter охватывает вакансии как российского рынка труда, так и предлагает заманчивые высокооплачиваемые должности в странах ближнего зарубежья, Европы и Азии.

DS-специалисты, за счет своей узкой специализации ценны и востребованы на рынке труда. Хотя подходящих вакансий для них значительно меньше (всего 3,6% от общего числа размещенных вакансий), но условия работы, возможности профессионального и карьерного роста и уровень дохода таких специалистов остаются на высоте.

## Дополнительный анализ работодателей и регионов присутствия

In [109]:
# сделаем выборки работодателей по числу размещенных ими активных вакансий

query_add_01 = '''select 
                    count(distinct employer_id)
                from 
                    public.vacancies
'''
df_add_01 = pd.read_sql_query(query_add_01, connection)
active_employers_ratio = df_add_01.iloc[0,0] / df_3_2.iloc[0,0] * 100
print(f'{df_add_01.iloc[0,0]} работодателей разместили вакансии в базе ({round(active_employers_ratio)}% от общего числа работодателей)')

query_add_02 = '''with employers_data as 
                        (select 
                            employer_id as employers, 
                            count(id) as vacancies_count
                        from 
                            public.vacancies
                        group by 
                            employer_id
                        having 
                            count(id) = 1
                        )
                    
                    select 
                        count(employers)
                    from 
                        employers_data

'''
df_add_02 = pd.read_sql_query(query_add_02, connection)
employers_ratio_1 = df_add_02.iloc[0,0] / df_add_01.iloc[0,0] * 100
print(f'{df_add_02.iloc[0,0]} работодателей разместили по 1 вакансии ({round(employers_ratio_1, 2)}% от активных работодателей)')

query_add_03 = '''with employers_data as 
                        (select 
                            employer_id as employers, 
                            count(id) as vacancies_count
                        from 
                            public.vacancies
                        group by 
                            employer_id
                        having 
                            count(id) > 1 and count(id) <= 10
                        )
                    
                    select 
                        count(employers)
                    from 
                        employers_data

'''
df_add_03 = pd.read_sql_query(query_add_03, connection)
employers_ratio_2_10 = df_add_03.iloc[0,0] / df_add_01.iloc[0,0] * 100
print(f'{df_add_03.iloc[0,0]} работодателей разместили от 2 до 10 вакансий ({round(employers_ratio_2_10, 2)}% от активных работодателей)')

query_add_04 = '''with employers_data as 
                        (select 
                            employer_id as employers, 
                            count(id) as vacancies_count
                        from 
                            public.vacancies
                        group by 
                            employer_id
                        having 
                            count(id) > 10 and count(id) <= 100
                        )
                    
                    select 
                        count(employers)
                    from 
                        employers_data

'''
df_add_04 = pd.read_sql_query(query_add_04, connection)
employers_ratio_11_100 = df_add_04.iloc[0,0] / df_add_01.iloc[0,0] * 100
print(f'{df_add_04.iloc[0,0]} работодателей разместили от 11 до 100 вакансий ({round(employers_ratio_11_100, 2)}% от активных работодателей)')

query_add_05 = '''with employers_data as 
                        (select 
                            employer_id as employers, 
                            count(id) as vacancies_count
                        from 
                            public.vacancies
                        group by 
                            employer_id
                        having 
                            count(id) > 100
                        )
                    
                    select 
                        count(employers)
                    from 
                        employers_data

'''
df_add_05 = pd.read_sql_query(query_add_05, connection)
employers_ratio_more_100 = df_add_05.iloc[0,0] / df_add_01.iloc[0,0] * 100
print(f'{df_add_05.iloc[0,0]} работодателей разместили более 100 вакансий ({round(employers_ratio_more_100, 2)}% от активных работодателей)')

query_add_06 = '''with employers_data as 
                        (select 
                            employer_id as employers, 
                            count(id) as vacancies_count
                        from 
                            public.vacancies
                        group by 
                            employer_id
                        having 
                            count(id) > 10
                        )
                    
                    select 
                        sum(vacancies_count)
                    from 
                        employers_data

'''
df_add_06 = pd.read_sql_query(query_add_06, connection)
vacancies_ratio = df_add_06.iloc[0,0] / df_3_1.iloc[0,0] * 100
print(f'{df_add_06.iloc[0,0]} вакансий размещены крупными работодателями (имеющими более 10 вакансий) ({round(vacancies_ratio)}% от всех вакансий в базе)')


14906 работодателей разместили вакансии в базе (63% от общего числа работодателей)
8699 работодателей разместили по 1 вакансии (58.36% от активных работодателей)
5582 работодателей разместили от 2 до 10 вакансий (37.45% от активных работодателей)
600 работодателей разместили от 11 до 100 вакансий (4.03% от активных работодателей)
25 работодателей разместили более 100 вакансий (0.17% от активных работодателей)
21206.0 вакансий размещены крупными работодателями (имеющими более 10 вакансий) (43% от всех вакансий в базе)


In [110]:
# Посчитаем, во скольких регионах из занесенных в базу, имеются активные вакансии

query_add_07 = '''select 
                    count(distinct area_id)
                from 
                    public.vacancies
'''
df_add_07 = pd.read_sql_query(query_add_07, connection)
active_area_ratio = df_add_07.iloc[0,0] / df_3_3.iloc[0,0] * 100
print(f'Активные вакансии размещены в {df_add_07.iloc[0,0]} регионах ({round(active_area_ratio)}% от общего числа регионов в базе)')

Активные вакансии размещены в 769 регионах (56% от общего числа регионов в базе)


In [111]:
# Найдем регионы, в которых предлагается самая высока зарплата

query_add_08 = '''select a.name as area_name, round(avg((coalesce(v.salary_from, v.salary_to) + coalesce(v.salary_to, v.salary_from))/2))
                from 
                    public.vacancies v
                    join public.areas a on a.id = v.area_id
                where salary_from is not null or salary_to is not null
                group by 
                    a.id 
                order by 2 desc
                '''
                
hh_avg_salary = pd.read_sql_query(query_add_08, connection)
area_list = list(hh_avg_salary['area_name'])
hh_avg_salary.head(20)

,area_name,round
0,Германия,452751.0
1,Болгария,324348.0
2,Испания,300000.0
3,Черногория,288589.0
4,Литва,263019.0
5,Польша,257156.0
6,Петропавловка (Республика Бурятия),250000.0
7,Турция,235435.0
8,Кипр,223736.0
9,Кутаиси,220475.0


### Выводы по анализу работодателей

По результатам предварительного анализа мы видим, что база данных содержит 49 197 вакансий и сведения о 23 501 работодателе. При этом, вакансии разместило лишь **14 906 компаний** (63% от всех имеющихся работодателей в базе).

Потребности работодателей в сотрудниках также сильно различаются:
* 58,36% активных работодателей (8699 компаний) ищут 1 сторудника (разместили по 1 вакансии);
* 37,45% активных работодателей (5582 компании) ищут от 2 до 10 сотрудников;
* 4,03% активных работодателей (600 компании) ищут от 11 до 100 сотрудников;
* 0,17% активных работодателей (25 компаний) требуется более 100 сотрудников.

При этом, на долю крупных работодателей (последние две категории) приходится **43% вакансий** из всех размещенных в базе.

### Выводы по анализу регионов присутствия

Предварительный анализ регионов показал, что всего в базе данных содержится информация о 1362 регионах. Однако текущие вакансии открыты лишь **по 769 регионам** (56% от имеющихся).

Анализ средней заработной платы по регионам показал, что самая высокая зарплата предлагается IT-специалистам за рубежом. 17 из ТОП 20 регионов по размеру заработной платы находятся за пределами России, а средний уровень зарплаты в этих регионах находится в диапазоне от 200000 рублей до 452751 рублей.

Чтобы проводить адекватный анализ вакансий для российских специалистов, следовало бы сперва разделить имеющующуюся базу данных на российские и зарубежные вакансии. Слеовательно изменились бы и все проверяемые показатели (число вакансий и работодателей, средняя зарплатная вилка, набор требований к соискателям).

Исходя из вышесказанного нужно отметить, что значения в признаке региона не унифицированы и могут принимать любые масштабы - страна, регион/область/край, город. В базе имеются вакансии и для России и для стран СНГ и для других зарубежных стран.

Логической разбивки для объединения городов по регионам и странам (от мелкого объекта к крупному и наоборот) системой не предусмотрено. А понять из таблицы, к какому типу географических объектов относится регион, очень сложно, особенно с учетом того, что географические названия могут повторяться.

Следовательно, основывать расчеты в данных на этом признаке можно лишь с учетом большой погрешности в результата. Либо нужно проводить серьезную работу по предобработке данного признака.

Также можно добавить, что встречаются и очень странные названия регионов, например "Яя", скорее всего это выбросы, ошибки ввода данных.  

***

## Дополнительный анализ по востребованным ключевым навыкам

Проведем анализ ключевых навыков, требуемых от соискателей. Для этого выделим из каждой вакансии список ключевых навыков и посчитаем по каждому навыку общее число его вхождений в базе данных.

In [112]:
# Выделим основные ключевые навыки, которые требуются от IT-специалиста

query_add_09 = '''select 
                    key_skills
                from 
                    public.vacancies
'''

key_skills = pd.read_sql_query(query_add_09, connection).dropna()
key_skills['key_skills'] = key_skills['key_skills'].apply(lambda x: x.split(sep='\t'))

key_skills_list = []
key_skills['key_skills'] = key_skills['key_skills'].apply(lambda x: key_skills_list.extend(x))
key_counter = Counter(key_skills_list)


key_skils_full = pd.DataFrame(key_counter.items(), columns=['key_skills', 'count']).sort_values('count', ascending=False, ignore_index=True)

top_skills = ', '.join(list(key_skils_full['key_skills'].head(40)))
print(top_skills)

SQL, Git, Английский язык, Работа в команде, Linux, JavaScript, Python, Управление проектами, 1С программирование, PostgreSQL, Грамотная речь, Аналитическое мышление, Java, 1С: Предприятие 8, ООП, HTML, MS PowerPoint, MySQL, PHP, Пользователь ПК, MS SQL, CSS, Работа с большим объемом информации, Настройка ПК, C#, Настройка сетевых подключений, Информационные технологии, Бизнес-анализ, Atlassian Jira, Организаторские навыки, Настройка ПО, Docker, Деловая коммуникация, Анализ данных, Разработка технических заданий, BPMN, Деловое общение, Техническая поддержка, 1С: Документооборот, Грамотность


In [113]:
# Выделим основные ключевые навыки, которые требуются от специалистов по работе с данными

query_add_10 = '''select 
                    key_skills
                from 
                    public.vacancies
                 where 
                    (name ilike '%data scientist%' 
                    or name ilike '%data science%' 
                    or name ilike '%исследователь данных%' 
                    or (name like '%ML%' and name not like '%HTML%') 
                    or name ilike '%machine learning%' 
                    or name ilike '%машинн%обучен%')
'''

key_skills_ds = pd.read_sql_query(query_add_10, connection).dropna()
key_skills_ds['key_skills'] = key_skills_ds['key_skills'].apply(lambda x: x.split(sep='\t'))

key_skills_ds_list = []
key_skills_ds['key_skills'] = key_skills_ds['key_skills'].apply(lambda x: key_skills_ds_list.extend(x))
key_counter_ds = Counter(key_skills_ds_list)


key_skils_ds = pd.DataFrame(key_counter_ds.items(), columns=['key_skills', 'count']).sort_values('count', ascending=False, ignore_index=True)

top_ds_skills = ', '.join(list(key_skils_ds['key_skills'].head(20)))
print(top_ds_skills)

Python, SQL, Machine Learning, Git, Математическая статистика, Data Analysis, Data Science, Pandas, ML, Английский язык, Linux, Анализ данных, PyTorch, Big Data, Математическое моделирование, Статистический анализ, NLP, C++, Spark, Tensorflow


### Выводы по анализу ключевых навыков

Можно выделить список наиболее востребованных навыков **для IT-сферы** вцелом. 

Привожу ТОП 40 навыков по популярности:
SQL, Git, Английский язык, Работа в команде, Linux, JavaScript, Python, Управление проектами, 1С программирование, PostgreSQL, Грамотная речь, Аналитическое мышление, Java, 1С: Предприятие 8, ООП, HTML, MS PowerPoint, MySQL, PHP, Пользователь ПК, MS SQL, CSS, Работа с большим объемом информации, Настройка ПК, C#, Настройка сетевых подключений, Информационные технологии, Бизнес-анализ, Atlassian Jira, Организаторские навыки, Настройка ПО, Docker, Деловая коммуникация, Анализ данных, Разработка технических заданий, BPMN, Деловое общение, Техническая поддержка, 1С: Документооборот, Грамотность.

Как мы видим, ключевые технические навыки включают в себя знание языков программирования (конкретный язык зависит от вакансии), работа с GIT и СУБД, навыки работы с ПК и операционными системами.

Также для работодателей важны и soft-скилы - знание английского языка, умение работы в команде, организаторские способности, граммотность и аналитическое мышление.

**Для дата сайентиста** список наиболее востребованных навыков несколько отличается, что логично учитывая специфику профессии.

Привожу ТОП 20 навыков по популярности:
Python, SQL, Machine Learning, Git, Математическая статистика, Data Analysis, Data Science, Pandas, ML, Английский язык, Linux, Анализ данных, PyTorch, Big Data, Математическое моделирование, Статистический анализ, NLP, C++, Spark, Tensorflow.

Как мы видим, работодатели при поиске сотрудника делают упор на технические навыки: владение Python, SQL, Git и знание теоретической базы (математической статистики и моделирования, статистического анализа).


***

## Дальнейшие направления для исследования

Предоставленныая база данных имеет существенный, связанный с некорректным учетом регионов размещения вакансий (писал об этом выше). 

В первую очередь, я бы провел **глубокую предобработку регионов** и сделал актуальные для исследования выборки: база по городам и регионам России, и база по зарубежным регионам. 

Кроме того, база данных содержит много пустых и некорректных значений (например, во многих вакансиях не указана вилка заработной платы, либо указаны явно заниженные цифры (30 рублей), не заполнены ключевые навыки или имеются не полные сведения о работодателе). 

Для подготовки базы к анализу нужна **предобработка некорректных признаков, очистка их от выбросов**, а где возможно заполнение пустых строк средними или медианными значениями.

После этого стоило бы пересчитать ключевые показатели по каждой из этих выборок в отдельности.

Самый важный критерий в базе, на мой взгляд - это **уровень зароботной платы**. Я бы поработал с ним детальнее и изучил его **в разрезе отдельных признаков**:
* средняя зарплата по российским регионам (Москва, Санкт-Петербург, города-миллионники, другие города);
* средняя зарплата по различным IT-специальностям (можно сделать выборки по ключевым словам из названия вакансии или по ключевым навыкам);
* средняя зарплата в зависимости от опыта работы, в том числе с разбивкой по различным IT-специальностям;
* средняя зарплата по отдельным работодателям и их сферам деятельности (для этого можно разделить работодателей на несколько групп, например, по их масштабу и числу востребованных специалистов).

Зарплата показывает ценность сотрудника, а следовательно имея данные по заработной плате в каждом срезе данных, мы можем выделить важные для работодателя и соискателя критерии оценки труда и квалификации специалиста. Зная это мы можем спрогнозировать подходящие вакансии для конкретных соискателей сервиса Headhunter.

